In [1]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import VectorStoreInfo
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)

D:\study\LammaIndex\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LLAMA_CLOUD_API_KEY"] = os.getenv("LLAMA_API_KEY")
os.environ["QDRANT_API_KEY"] = os.getenv("QDRANT_API_KEY")

In [3]:
# Cấu hình client Qdrant
client = QdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)

# Cấu hình LLM và Embedding
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = OpenAI(model="gpt-4o-mini")

In [11]:
# Khởi tạo Vector Store
vector_store = QdrantVectorStore(
    collection_name="thong_tin_san_pham",
    client=client,
    aclient=aclient,
)
filters = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.IN, value=["NetSure_732_User_Manual", "NetSure_732_Brochure"]),
        MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
    ],
    condition=FilterCondition.AND,
)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

retriever = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters)

In [12]:
# --- Thay đổi từ đây ---
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
)

In [17]:
query_str = f"""tiêu chuẩn IEC60950-1"""

In [18]:
# 2. Thực hiện truy vấn qua Query Engine
print("Bắt đầu truy vấn với Query Engine...")
response = query_engine.retrieve(query_str)

Bắt đầu truy vấn với Query Engine...


In [19]:
for node in response:
    print(node.metadata)
    print(node.get_content())

{'file_name': 'NetSure_732_User_Manual', 'product_name': 'NetSure 732', 'header_path': '/Appendix 1   Technical And Engineering Data/', 'page': 29, 'level1': 'Appendix 1   Technical And Engineering Data', 'level2': 'Appendix 1    Technical And Engineering Data', 'summary': 'The text provides detailed technical and engineering data for the NetSure 732 Subrack Power System, including parameters related to DC output, rectifier specifications, EMC compliance, and safety regulations.', 'table_name': 'Technical and Engineering Data for NetSure 732 Subrack Power System', 'figure_name': 'None', 'document_id': '9c90c4b6-6acd-11f0-be9a-d21c9e6a59d7', 'chunk_id': 'NetSure_732_User_Manual_92', 'type': 'chunk_document'}
Lock out at the second over-voltage<br/>When the output voltage reaches the software protection point, the rectifier will shut down, and restart automatically after 5 seconds. If the over-voltage happens again within a set time (default: 5min. Configurable through controller), the r